# `tf.io` 模块
主要用于 数据输入输出（I/O），即数据的 读取、解析、序列化、编码、解码 等操作。它在数据处理管道（特别是 `tf.data`）中经常用到，常用于 TFRecord 文件的读写、字符串处理、序列化/反序列化、编码转换。

## 一、常见功能分类
### 1. TFRecord 文件读写

`tf.io.TFRecordWriter(path)`

   - 用于写入 TFRecord 文件。

`tf.io.tf_record_iterator(path)`

   - 用于遍历 TFRecord 文件中的序列化 Example。

### 2. Example/Feature 序列化 & 反序列化

`tf.io.serialize_tensor(tensor)`

  - 将张量序列化成字符串。

`tf.io.parse_tensor`(serialized, out_type)

  - 将序列化的字符串反序列化回张量。

`tf.io.parse_single_example(serialized, features)`

  - 解析单个序列化的 `tf.train.Example`。

`tf.io.parse_example(serialized, features)`

  - 解析批量的 `Example`。

### 3. 数据编码与解码

`tf.io.encode_jpeg(image)` / `tf.io.decode_jpeg(contents)`

 - 图像编码/解码。

`tf.io.encode_png(image)` / `tf.io.decode_png(contents)`

`tf.io.encode_base64(input)` / `tf.io.decode_base64(input)`

 - Base64 编解码（常用于网络数据存储）。

`tf.io.decode_raw(bytes, out_type)`

 - 将原始字节解码为张量。

### 4. 字符串与字节处理

`tf.io.serialize_tensor(tensor)`

`tf.io.parse_tensor(serialized, out_type)`

`tf.io.decode_compressed(bytes, compression_type)`

- 解压缩数据（如 GZIP）。

## 二、常用示例
### 1. 写入和读取 TFRecord

In [ ]:
import tensorflow as tf

# 写入
with tf.io.TFRecordWriter("data.tfrecord") as writer:
    for i in range(3):
        feature = {
            "x": tf.train.Feature(int64_list=tf.train.Int64List(value=[i])),
            "y": tf.train.Feature(float_list=tf.train.FloatList(value=[i * 0.5]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())

# 读取
raw_dataset = tf.data.TFRecordDataset("data.tfrecord")
for raw_record in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)


### 2. 序列化 & 反序列化张量

In [2]:
import tensorflow as tf 
tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
serialized = tf.io.serialize_tensor(tensor)
print("序列化:", serialized)

restored = tf.io.parse_tensor(serialized, out_type=tf.float32)
print("还原:", restored)


序列化: tf.Tensor(b'\x08\x01\x12\x08\x12\x02\x08\x02\x12\x02\x08\x02"\x10\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@', shape=(), dtype=string)
还原: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


### 3. 图像编码与解码

In [ ]:
import tensorflow as tf

image = tf.random.uniform(shape=[64, 64, 3], minval=0, maxval=255, dtype=tf.int32)
image = tf.cast(image, tf.uint8)

# 编码为 JPEG 格式
jpeg_bytes = tf.io.encode_jpeg(image)

# 解码回张量
decoded = tf.io.decode_jpeg(jpeg_bytes)

print("原图:", image.shape, "解码后:", decoded.shape)


## 三、参数表格示例（以 tf.io.parse_single_example 为例）
| 参数           | 类型                      | 说明                                                                          |
| ------------ | ----------------------- | --------------------------------------------------------------------------- |
| `serialized` | `Tensor` (dtype=string) | 单个序列化的 Example（二进制字符串）                                                      |
| `features`   | dict                    | 指定要解析的特征结构，key 是特征名，value 是 `tf.io.FixedLenFeature` 或 `tf.io.VarLenFeature` |

---
## 四、总结

- `tf.io` = TensorFlow 的数据 I/O 工具箱

- 常用场景：

   - 1、TFRecord 文件存储和读取

   - 2、张量的序列化/反序列化

   - 3、数据的编码/解码（JPEG、PNG、Base64、压缩）

   - 4、解析 Example 格式数据